In [1]:
! pip install transformers peft accelerate datasets evaluate
! pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
import os
import peft
import torch
import torch.nn as nn
import evaluate
import transformers
import argparse # Parsing command line arguments
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from peft import get_peft_config,get_peft_model,get_peft_model_state_dict,set_peft_model_state_dict,LoraConfig,PeftType,PrefixTuningConfig,PromptEncoderConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer,get_linear_schedule_with_warmup,set_seed


In [7]:
batch_size=32
model_name='roberta-large'
task='mrpc'
peft_type=PeftType.LORA
device='cuda'
num_epochs=20

In [8]:
peft_config=LoraConfig(task_type='SEQ_CLS',inference_mode=False,r=8,lora_alpha=16,lora_dropout=0.1)
lr=3e-4

In [9]:
if any(k in model_name for k in ('gpt','opt','bloom')):
  padding_side='left'
else:
  padding_side='right'

tokenizer=AutoTokenizer.from_pretrained(model_name,padding_side=padding_side)
# Add pad_token at end of sentence
if getattr(tokenizer,'pad_token_id') is None:
  tokenizer.pad_token_id=tokenizer.eos_token_id

datasets=load_dataset('glue',task)
metric=evaluate.load('glue',task)

def tokenize_function(examples):
  outputs=tokenizer(examples['sentence1'],examples['sentence2'],truncation=True,max_length=None)
  return outputs

tokenized_datasets=datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['idx','sentence1','sentence2']
)

# Rename Label
tokenized_datasets = tokenized_datasets.rename_column('label','labels')

def collate_fn(examples):
  return tokenizer.pad(examples,padding='longest',return_tensors='pt')

# DataLoader
train_dataloader=DataLoader(tokenized_datasets['train'],shuffle=True,collate_fn=collate_fn,batch_size=batch_size)
eval_dataloader=DataLoader(tokenized_datasets['validation'],shuffle=False,collate_fn=collate_fn,batch_size=batch_size)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
# We are creating LoRa adapter in LLM
model=AutoModelForSequenceClassification.from_pretrained(model_name,return_dict=True)
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()
model

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,838,082 || all params: 357,199,876 || trainable%: 0.5146


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [11]:
# We Initialize the Optimizer and Learning rate
optimizer=AdamW(params=model.parameters(),lr=lr)

# Starts with small learning rate then linearly increase during warmup
lr_scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs), # Num training steps
    num_training_steps=(len(train_dataloader)* num_epochs) # total number of optimization steps
)

In [13]:
model.to(device)

# Model Training
for epoch in range(num_epochs):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader)):
    batch.to(device)
    outputs=model(**batch)
    loss= outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  # Model evaluation
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
      outputs= model(**batch)
    predictions=outputs.logits.argmax(dim=-1)
    predictions,references=predictions,batch['labels']
    metric.add_batch( predictions=predictions,references=references)

  eval_metric=metric.compute()
  print(f"Epoch{epoch}:",eval_metric)



100%|██████████| 13/13 [00:05<00:00,  2.22it/s]


Epoch0: {'accuracy': 0.7132352941176471, 'f1': 0.8245877061469266}


100%|██████████| 13/13 [00:05<00:00,  2.26it/s]


Epoch1: {'accuracy': 0.7230392156862745, 'f1': 0.8274809160305343}


100%|██████████| 13/13 [00:05<00:00,  2.24it/s]


...

Epoch17: {'accuracy': 0.8897058823529411, 'f1': 0.9197860962566845}


100%|██████████| 13/13 [00:05<00:00,  2.27it/s]


Epoch18: {'accuracy': 0.8823529411764706, 'f1': 0.9154929577464789}


100%|██████████| 13/13 [00:05<00:00,  2.27it/s]

Epoch19: {'accuracy': 0.8823529411764706, 'f1': 0.9148936170212766}


In [14]:
# Share to HUB
model.push_to_hub('OmBhandwalkar/roberta-large-peft-lora',use_auth_tokens=True)

adapter_model.safetensors:   0%|          | 0.00/7.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/roberta-large-peft-lora/commit/f7c7c1ad96536a625eb748b4b91f92e03f54d864', commit_message='Upload model', commit_description='', oid='f7c7c1ad96536a625eb748b4b91f92e03f54d864', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/roberta-large-peft-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/roberta-large-peft-lora'), pr_revision=None, pr_num=None)

In [16]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "OmBhandwalkar/roberta-large-peft-lora"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

adapter_config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/7.37M [00:00<?, ?B/s]

100%|██████████| 13/13 [00:05<00:00,  2.39it/s]

{'accuracy': 0.8823529411764706, 'f1': 0.9148936170212766}
